In [ ]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
df = pd.read_csv('movies.csv')

## Basic Overview of dataset

In [ ]:
df.head()

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
df.columns

## Extracting id and respective poster_path
this will be helpful in showing posters along with recommended movie names.

In [ ]:
poster_paths = df[['id', 'poster_path']]

In [ ]:
pickle.dump(poster_paths, open('poster_paths.pkl', 'wb'))

## Filtering columns 
Only keeping those columns which will help in building recommender system.

In [ ]:
df['original_language'].value_counts()

In [ ]:
df[df['original_language'] == 'en'].shape[0]/df.shape[0]
# more than 75% of movies are in english so not important column

In [ ]:
df['adult'].value_counts()

### columns to include
- id
- title
- overview
- genre_names 
- keywords
- cast 
- director

In [ ]:
df2 = df.copy()

In [ ]:
df = df[['id', 'title', 'overview', 'genre_names', 'keywords', 'cast', 'director']]

## Handling duplicate values

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df[['title', 'director']].duplicated().sum()

In [ ]:
df[['title', 'director']].drop_duplicates(inplace=True)

## Handling missing values

In [ ]:
df.isna().sum()

In [ ]:
df[df['title'].isna()]

In [ ]:
df[df['genre_names'].isna()]

In [ ]:
df[df['keywords'].isna()]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

## Formatting textual columns into list

In [ ]:
df['genre_names'][0]

In [ ]:
ast.literal_eval("['Action', 'Comedy', 'Science Fiction']")

In [ ]:
df['keywords'][0]

In [ ]:
df['cast'][0]

In [ ]:
df['director'][0]

In [ ]:
def create_list(name):
    return [name]

### Creating lists from all the textual columns

In [ ]:
df['genre_names'] = df['genre_names'].apply(ast.literal_eval)
df['keywords'] = df['keywords'].apply(ast.literal_eval)
df['cast'] = df['cast'].apply(ast.literal_eval)
df['director'] = df['director'].apply(create_list)
df['overview'] = df['overview'].apply(lambda x:x.split())

### Removing spaces between words so that meaning remains

In [ ]:
def collapse(L):
    l = []
    for i in L:
        l.append(i.replace(' ', ''))
    return l

In [ ]:
df['genre_names'].apply(collapse)

In [ ]:
df['keywords'].apply(collapse)

In [ ]:
df['cast'].apply(collapse)

In [ ]:
df['overview'] = df['overview'].apply(collapse)
df['genre_names'] = df['genre_names'].apply(collapse)
df['keywords'] = df['keywords'].apply(collapse)
df['cast'] = df['cast'].apply(collapse)
df['director'] = df['director'].apply(collapse)

In [ ]:
df['tags'] = df['overview'] + df['genre_names'] + df['keywords'] + df['cast'] + df['director']

### Creating new dataframe by dropping columns which are already included into tags

In [ ]:
new_df = df.drop(columns=['overview', 'genre_names', 'keywords', 'cast', 'director'])

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('stopwords')

In [ ]:
stemmer = PorterStemmer()

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = text.split()
    processed_words = [stemmer.stem(word) for word in words if word.lower() not in stop_words]
    return ' '.join(processed_words)

In [ ]:
new_df['processed_tags'] = new_df['tags'].apply(preprocess_text)

## Vectorizing strings

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['processed_tags']).toarray()

In [ ]:
vector

In [ ]:
vector.shape

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x:x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Deadpool & Wolverine')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))

### Processing for fetching poster path

In [ ]:
poster_paths[poster_paths['id'] == '704239']['poster_path'].values[0]

In [ ]:
new_df[new_df['id'] == '704239']